In [ ]:
!pip install fastapi uvicorn pyngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Preprocessing

In [ ]:
%%writefile preprocessing.py
import re
import os
import pandas as pd

# KONFIG
MAX_LEN = 30

# PATH KAMUS SLANG (CSV)
KAMUS_PATH = "/content/drive/MyDrive/Colab Notebooks/Deep learning/TUBES/nonbaku_gabungan_kamus.csv"

# LOAD SLANG DICTIONARY
def load_slang_dictionary(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Kamus slang tidak ditemukan: {path}")

    df = pd.read_csv(path)
    df.columns = df.columns.str.lower().str.strip()

    if 'slang' not in df.columns or 'formal' not in df.columns:
        raise ValueError("CSV harus punya kolom 'slang' dan 'formal'")

    slang_dict = dict(
        zip(
            df['slang'].astype(str),
            df['formal'].astype(str)
        )
    )

    print(f"✅ Slang dictionary loaded: {len(slang_dict)} entries")
    return slang_dict


slang_dict = load_slang_dictionary(KAMUS_PATH)

# FINAL PREPROCESSING FUNCTION
def preprocess_complete(text):
    if not isinstance(text, str):
        return ""

    # lowercase
    text = text.lower()

    # hapus token username & URL
    text = re.sub(r"<username>", " ", text)
    text = re.sub(r"http\S+|www\S+", " ", text)

    # hapus mention & hashtag
    text = re.sub(r"@\w+", " ", text)
    text = re.sub(r"#", " ", text)

    # normalisasi elongasi (anjiiirrr → anjir)
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # hapus karakter non huruf
    text = re.sub(r"[^a-z\s]", " ", text)

    # rapikan spasi
    text = re.sub(r"\s+", " ", text).strip()

    # normalisasi slang
    words = text.split()
    words = [slang_dict.get(w, w) for w in words]

    return " ".join(words)

#API

In [ ]:
%%writefile api.py

from fastapi import FastAPI
from pydantic import BaseModel
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

from preprocessing import preprocess_complete, MAX_LEN


# LOAD MODEL
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/Deep learning/TUBES/Model/denganfasttext.h5"
TOKENIZER_PATH = "/content/drive/MyDrive/Colab Notebooks/Deep learning/TUBES/Model/tokenizer (3).pkl"

model = tf.keras.models.load_model(MODEL_PATH)

with open(TOKENIZER_PATH, "rb") as f:
    tokenizer = pickle.load(f)


# FASTAPI INIT
app = FastAPI(
    title="Cyberbullying Detection API",
    description="API untuk deteksi cyberbullying menggunakan BiLSTM",
    version="1.0"
)

class TextInput(BaseModel):
    text: str

@app.get("/")
def root():
    return {
        "status": "API running"
    }

@app.post("/predict")
def predict(data: TextInput):
    try:
        raw_text = data.text
        clean_text = preprocess_complete(raw_text)

        if not clean_text.strip():
            return {
                "input": raw_text,
                "label": "unknown",
                "score": 0.0,
                "note": "Empty after preprocessing"
            }

        seq = tokenizer.texts_to_sequences([clean_text])
        pad = pad_sequences(seq, maxlen=MAX_LEN, padding="post")

        score = float(model.predict(pad, verbose=0)[0][0])
        label = "cyberbullying" if score > 0.5 else "non-cyberbullying"

        return {
            "input": raw_text,
            "clean_text": clean_text,
            "label": label,
            "score": score
        }

    except Exception as e:
        return {"error": str(e)}

#Run ngrok+api

In [ ]:
from pyngrok import ngrok
import threading
import uvicorn


# SET TOKEN
ngrok.set_auth_token("362ocoEjtgGftMd2Z8bWNbBUkQh_3jVryS1qitHi7UEdsHfuK")
PORT = 8000


# OPEN NGROK TUNNEL
public_url = ngrok.connect(PORT)
print("🚀 Public URL:", public_url)


# RUN FASTAPI
def run():
    uvicorn.run("api:app", host="0.0.0.0", port=PORT, log_level="info")

threading.Thread(target=run).start()

🚀 Public URL: NgrokTunnel: "https://nongregariously-unabdicative-kym.ngrok-free.dev" -> "http://localhost:8000"
